In [ ]:
%pip install paramiko

In [ ]:
import paramiko
from google.cloud import storage
import os

In [ ]:
import os
import paramiko
from google.cloud import storage

def ssh_to_cloudera_server():
    """
    Downloads a private key from GCS and uses it to establish an SSH connection.
    """
    # --- Configuration Details ---
    GCS_KEY_URI = "gs://workbench-essentials/cloudera-connect/x.pem"
    SSH_HOST = "10.254.134.3"
    SSH_USER = "Lukas.Hanzlik"

    # The temporary path where the key will be saved
    LOCAL_KEY_PATH = "/tmp/ssh_key.pem"

    try:
        # 1. Download the private key from GCS
        print("📥 Attempting to download the private key from GCS...")
        storage_client = storage.Client()
        bucket_name, blob_name = GCS_KEY_URI.replace("gs://", "").split("/", 1)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.download_to_filename(LOCAL_KEY_PATH)

        # Set file permissions to be read-only for the owner
        os.chmod(LOCAL_KEY_PATH, 0o600)
        print("✅ Private key downloaded and saved with correct permissions.")

        # 2. Establish the SSH connection
        print(f"🔗 Attempting to establish an SSH connection to {SSH_USER}@{SSH_HOST}...")
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Load the private key for authentication
        private_key = paramiko.RSAKey.from_private_key_file(LOCAL_KEY_PATH)
        ssh_client.connect(hostname=SSH_HOST, username=SSH_USER, pkey=private_key)

        print("🎉 SSH connection established successfully!")

        # 3. Execute a sample command to verify the connection
        print("\n--- Listing directories on the remote server ---")
        stdin, stdout, stderr = ssh_client.exec_command('ls -l /')
        print(stdout.read().decode())

    except paramiko.ssh_exception.AuthenticationException:
        print("❌ Authentication failed. Make sure your username and private key are correct.")
    except Exception as e:
        print(f"❌ An error occurred during the SSH connection: {e}")
    finally:
        # 4. Clean up the temporary key file
        if os.path.exists(LOCAL_KEY_PATH):
            os.remove(LOCAL_KEY_PATH)
            print("🗑️ The temporary key file has been deleted.")

        # Ensure the SSH connection is closed
        if 'ssh_client' in locals() and ssh_client.get_transport() is not None and ssh_client.get_transport().is_active():
            ssh_client.close()

# Run the function
ssh_to_cloudera_server()